In [1]:
from google.colab import files
uploaded = files.upload()

Saving mnist_test.csv to mnist_test.csv
Saving mnist_train.csv to mnist_train.csv
Saving mnist_test_with_labels.csv to mnist_test_with_labels.csv


In [0]:
import tensorflow as tf
import pandas as pd
import io
import numpy as np

In [0]:
def one_hot_fn(values):
  
  ret = []
  for value in values:
    tmp = np.zeros(10)
    tmp[value] = 1
    ret.append(tmp)
  return (np.array(ret,dtype=int))

In [0]:
train = pd.read_csv(io.StringIO(uploaded['mnist_train.csv'].decode('utf-8')))

train_labels = pd.DataFrame()
train_labels['Category'] = train['Category']
train_values = train.drop(['Id', 'Category'], axis=1)

train_val = np.array(train_values.values.tolist(), dtype=int)
train_lab = np.array(train_labels.values.tolist(), dtype=int)

train_labe = one_hot_fn(train_lab)

train_val1 = train_val.copy()

for i in range(len(train_val1)):
  for j in range(len(train_val1[i])):
    if train_val1[i][j] >= 50:
      train_val1[i][j] = 1
    else:
      train_val1[i][j] = 0
    
test = pd.read_csv(io.StringIO(uploaded['mnist_test_with_labels.csv'].decode('utf-8')))
test_values = np.array(test.drop(['Id','Category'], axis=1).values.tolist(), dtype=int)

test_val = test_values.copy()

for i in range(len(test_val)):
  for j in range(len(test_val[i])):
    if test_val[i][j] >= 50:
      test_val[i][j] = 1
    else:
      test_val[i][j] = 0
    
test_labels = np.array(test['Id'].values.tolist(), dtype=int)
test_lab = one_hot_fn(test_labels)

test1 = pd.read_csv(io.StringIO(uploaded['mnist_test.csv'].decode('utf-8')))
test_df = test1.drop(['Id'], axis=1)
test_values1 = np.array(test_df.values.tolist(), dtype=int)

test_val1 = test_values1.copy()

for i in range(len(test_val1)):
  for j in range(len(test_val1[i])):
    if test_val1[i][j] >= 50:
      test_val1[i][j] = 1
    else:
      test_val1[i][j] = 0

In [0]:
learning_rate = 0.25
epochs = 15
batch_size = 100
dropout = 0.5

In [0]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32) 

W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')

W2 = tf.Variable(tf.random_normal([300, 100], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([100]), name='b2')

W3 = tf.Variable(tf.random_normal([100, 10], stddev=0.03), name='W3')
b3 = tf.Variable(tf.random_normal([10]), name='b3')

In [0]:
hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

hidden_out1 = tf.add(tf.matmul(hidden_out, W2), b2)
hidden_out1 = tf.nn.relu(hidden_out1)

layerdrop = tf.nn.dropout(hidden_out1, keep_prob)

y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out1, W3), b3))

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                         + (1 - y) * tf.log(1 - y_clipped), axis=1))

optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

init_op = tf.global_variables_initializer()

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [41]:
with tf.Session() as sess:
  sess.run(init_op)
  total_batch = int(len(test_lab) / batch_size)
  for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x = test_val[i*batch_size:(i+1)*batch_size] 
            batch_y = test_lab[i*batch_size:(i+1)*batch_size]
            _, c = sess.run([optimiser, cross_entropy], 
                         feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
  print(sess.run(accuracy, feed_dict={x: test_val, y: test_lab, keep_prob: dropout}))
  predicted = sess.run(tf.argmax(y_, 1), feed_dict={x: test_val1})

print(predicted)
full_mat = []
for i in range(len(predicted)):
  full_mat.append([i+1, predicted[i]])

labels = ['Id', 'Category']
new_df = pd.DataFrame.from_records(full_mat, columns=labels)
new_df.to_csv('predictions.csv',index=False)
#print(new_df.head(5))
#files.download('predictions.csv')

Epoch: 1 cost = 2.134
Epoch: 2 cost = 0.703
Epoch: 3 cost = 0.435
Epoch: 4 cost = 0.418
Epoch: 5 cost = 0.323
Epoch: 6 cost = 0.221
Epoch: 7 cost = 0.152
Epoch: 8 cost = 0.113
Epoch: 9 cost = 0.106
Epoch: 10 cost = 0.678
Epoch: 11 cost = 0.206
Epoch: 12 cost = 0.089
Epoch: 13 cost = 0.050
Epoch: 14 cost = 0.031
Epoch: 15 cost = 0.019
0.99639964
[7 2 1 ... 4 5 6]


In [91]:
print(len(test_val1))

10000


In [0]:
files.download('predictions.csv')